# Agentic RAG

이번 챕터에서는 문서 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성하는 에이전트를 만들어 보겠습니다.

질문에 따라 문서를  검색하여 답변하거나,인터넷 검색 도구를 활용하여 답변하는 에이전트를 만들어 보겠습니다.

**참고**

- RAG 를 수행하되, Agent 를 활용하여 RAG 를 수행한다면 이를 **Agentic RAG** 라고 부릅니다.

## 도구(Tools)

Agent 가 활용할 도구를 정의하여 Agent 가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있습니다.

Tavily Search 는 그 중 대표적인 **검색 도구** 입니다. 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성할 수 있습니다. 도구는 이처럼 검색 도구 뿐만아니라 Python 코드를 실행할 수 있는 도구, 직접 정의한 함수를 실행하는 도구 등 다양한 종류와 방법론을 제공합니다.


In [1]:
import os
import sys
import json
import time
import requests

from dotenv import load_dotenv

def setup_env():
    
    env_path = os.path.join(os.getcwd(), '../.env')

    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        
        print(f"Loaded environment variables from: \033[94m{env_path}\033[0m")
    else:
            print("\033[91mError: .env file not found. Please create one with your OPENAI_API_KEY.\033[0m")
            sys.exit(1)

setup_env()

Loaded environment variables from: /home/ras/0.agent_ai_ws/src/learn_rag_and_agent/learn_rag_and_agent/../.env


In [2]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

In [3]:
# 검색 결과를 가져옵니다.
search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")

[{'title': '[판교]판교역 카카오 아지트 카페, 플링크, 휘낭시에맛집, 판교카페추천 ...',
  'url': 'https://blog.naver.com/PostView.nhn?blogId=songwon93&logNo=224073364654&redirect=Dlog&widgetTypeCall=true',
  'content': '| 카페/빵  [판교]판교역 카카오 아지트 카페, 플링크, 휘낭시에맛집, 판교카페추천,내돈내산  보라   ・  2025. 11. 12. 12:23  URL 복사  이웃추가 본문 기타 기능 공유하기  신고하기  위치플링크 판교  [판교]판교역 카카오 아지트 카페, 플링크, 휘낭시에맛집, 판교카페추천,내돈내산  2025년 11월  \u200b  판교 오피스로 출근하고 모닝 커피 겸 잠시 티타임하러 플링크에 갔다.  사내커피 가려구 했으나...그녀의 선호가 어쩔 수 없었다ㅋㅋ 바로 따라감.  ---  플링크 판교점  전화번호 | 0507-1428-7222  주소 | 경기 성남시 분당구 판교역로 166 1층 12호  영업시간 | 월~일 08:00-20:00  \u200b  ---  \u200b  플링크 판교 경기도 성남시 분당구 판교역로 166 1층 12호  \u200b  위치는 판교 카카오 아지트 1층에 위치해있다.  참고로 아지트로 들어오는 문이 꽤 많으므로, 1층에서 헤맬 수도 있다.  카카오 프렌즈샵 근처라고',
  'score': 0.8790744},
 {'title': '카카오프렌즈 판교 아지트점 판교의 카카오프렌즈를 소개합니다.',
  'url': 'https://m.blog.naver.com/hj961030/222923648353',
  'content': '본문 바로가기\n\n# 블로그\n\n## 카테고리 이동 ❤️늘이의 여행일기❤️\n\n검색\n\n카카오프렌즈 판교 아지트점 판교의 카카오프렌즈를 소개합니다.\n\nn2u1 늘이\n\n2022. 11. 8. 21:39\n\n이웃추가\n\n 본문 폰트 크기 조정 가\n 

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

In [5]:
# 문서에서 관련성 높은 문서를 가져옵니다.
retriever.invoke("삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘")

[Document(id='807456e9-1639-4675-b2e2-ec40ea567811', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'author': 'dj', 'moddate': '2023-12-08T13:28:38+09:00', 'pdfversion': '1.4', 'source': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'page': 12, 'page_label': '13'}, page_content='SPRi AI Brief |  \n2023-12 월호\n10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개\nn삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스 ’를 공개\nn삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents\n£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원\nn삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스 ’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 \n단계적으로 탑재할 계획\

In [6]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

In [7]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

## Agent 생성

이제 도구를 정의했으니 에이전트를 생성할 수 있습니다. 

먼저, Agent 가 활용할 LLM을 정의하고, Agent 가 참고할 Prompt 를 정의합니다.

**참고**
- 멀티턴 대화를 지원하지 않는다면 "chat_history" 를 제거해도 좋습니다.

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [9]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

In [10]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [11]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [13]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "2025년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2025년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2025년 프로야구 플레이오프 진출 팀'}`



[관찰 내용]
Observation: [{'title': '2025년 KBO 포스트시즌', 'url': 'https://ko.wikipedia.org/wiki/2025%EB%85%84_KBO_%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C', 'content': '여기를 가리키는 문서\n 가리키는 글의 최근 바뀜\n 파일 올리기\n 고유 링크\n 문서 정보\n 이 문서 인용하기\n 축약된 URL 얻기\n QR코드 다운로드\n\n인쇄/내보내기\n\n 책 만들기\n PDF로 다운로드\n 인쇄용 판\n\n다른 프로젝트\n\n보이기\n\n위키백과, 우리 모두의 백과사전.\n\n2025 신한 SOL Bank KBO 포스트시즌은 2025년 10월 6일부터 10월 31일까지 진행됐다. 이번 포스트시즌부터 한국시리즈 방식이 2-2-3에서 2-3-2로 다시 회귀하면서 한국시리즈 5차전 장소는 플레이오프 승리팀의 홈구장인 대전한화생명볼파크에서 치러진다.\n\n## 진출팀\n\n[편집]\n\n2025년 포스트시즌에 진출한 팀은 LG 트윈스, 한화 이글스, SSG 랜더스, 삼성 라이온즈, NC 다이노스다.\n\n LG 트윈스 [...] :   2019년부터 7년 연속으로 포스트시즌 진출에 성공했다. 지난 해에는 정규 시즌 3위로 마무리했으나 10월 1일, 매직넘버가 모두 소멸되면서 2023년 이후 2년만에 정규 시즌 우승을 차지해 한국시리즈에 직행했다.\n\n 한화 이글스\n\n:   2018년 이후 7년만에 포스트시즌 진출에 성공했다. 9월 18일 정규 시즌 2위를 확보하며 2007년 이후 18년만에 최소 플레이오프 직행을 확정했으며, 마지막까지 LG 트윈스와 정규

In [14]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "삼성전자가 자체 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스 ’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는  
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며 , 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이 (code.i)’ 는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원

## 이전 대화내용 기억하는 Agent


In [15]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [16]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "삼성전자가 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스 ’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는  
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며 , 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이 (code.i)’ 는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원

In [17]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
Here is the translation of the previous response into English:

1. **Samsung Gauss**: On November 8, 2023, Samsung Electronics unveiled its generative AI model called 'Samsung Gauss' at the 'Samsung AI Forum 2023'. This model consists of three components: language, code, and image, and is designed to operate on-device.

2. **On-Device Operation**: Samsung Gauss has the advantage of not risking the leakage of user information externally. It has been trained on secure data and developed in a way that does not infringe on licenses or personal information.

3. **Model Composition**:
   - **Language Model**: Supports tasks such as email writing, document summarization, and translation.
   - **Code Model**: Provides a conversational interface through the AI coding assistant 'code.i', optimized for in-house software development.
   - **Image Model**: Capable of generating creative images and modifying existing ones, as well as converting low-resolution images to high-resolution.

4. *

## Agent 템플릿

다음은 전체 템플릿 코드 입니다.


In [18]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

########## 1. 도구를 정의합니다 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

########## 2. LLM 을 정의합니다 ##########
# LLM 모델을 생성합니다.
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의합니다 ##########

# Prompt 를 정의합니다 - 이 부분을 수정할 수 있습니다!
# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의합니다 ##########

# 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의합니다 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가합니다. ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()

ValueError: File path data/SPRI_AI_Brief_2023년12월호_F.pdf is not a valid file or url

In [19]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "구글이 앤스로픽에 투자한 금액을 문서에서 찾아줘"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: Google Anthropic investment amount
Log: 
Invoking: `pdf_search` with `{'query': 'Google Anthropic investment amount'}`



[관찰 내용]
Observation: n구글은 수익률이 높은 클라우드 컴퓨팅 시장에서 아마존과 마이크로소프트를 따라잡고자 생성 AI를 
통한 기업 고객의 클라우드 지출 확대를 위해 AI 투자를 지속  
∙구글은 앤스로픽 외에도 AI 동영상 제작 도구를 개발하는 런웨이 (Runway) 와 오픈소스 소프트웨어 
기업 허깅 페이스 (Hugging Face) 에도 투자
∙구글은 챗GPT의 기반 기술과 직접 경쟁할 수 있는 차세대 LLM ‘제미니 (Gemini)’ 를 포함한 자체 AI 
시스템 개발에도 수십억 달러를 투자했으며 , 2024 년 제미니를 출시할 계획
☞ 출처 : The Wall Street Journal, Google Commits $2 Billion in Funding to AI Startup Anthropic, 2023.10.27.
Bloomberg, AI Startup Anthropic to Use Google Chips in Expanded Partnership, 2023.11.09.

18영국 옥스퍼드 인터넷 연구소 , AI 기술자의 임금이 평균 21% 높아
n옥스퍼드 인터넷 연구소의 연구에 따르면 특정 기술의 경제적 가치는 다른 기술과 결합 
가능성이 높을수록 높게 평가됨 
nAI의 확산은 기술의 경제적 가치에 크게 영향을 미치며 , AI 기술을 가진 근로자는 평균 21%, 
최대 40% 높은 임금을 받을 수 있음  KEY Contents
£AI 기술 중 머신러닝 , 텐서플로우 , 딥러닝의 임금 프리미엄이 높게 평가
n옥스퍼드 인터넷 연구소 (Oxford Internet Institute) 가 2023년 10월 24일 962개 기술과 2만 5천 
명을 대상으로 한 연구에서 

In [20]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
Here is the translation of the previous response into English:

Google has committed $2 billion in funding to the AI startup Anthropic. This investment is part of Google's ongoing efforts to expand its AI capabilities and compete in the cloud computing market.


In [21]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {
        "input": "2024년 프로야구 플레이오프 진출 5개팀을 검색해서 알려주세요. 한글로 답변하세요"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`



[관찰 내용]
Observation: [{'title': '프로야구 2024 포스트시즌 일정과 진행 방식 정리(각 팀별 포스트시즌 ...', 'url': 'https://pathfollower.tistory.com/entry/%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-2024-%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C-%EC%9D%BC%EC%A0%95%EA%B3%BC-%EC%A7%84%ED%96%89-%EB%B0%A9%EC%8B%9D-%EC%A0%95%EB%A6%AC%EA%B0%81-%ED%8C%80%EB%B3%84-%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C-%EC%A7%84%EC%B6%9C-%ED%99%95%EB%A5%A0?category=1211696', 'content': '플레이오프는 준플레이오프 승리팀과 정규시즌 2위 팀이 맞붙는 대결이다. 5판 3선승으로 진행된다.  \n1,2,5경기는 정규리그 2위 팀 홈구장에서 열리며 3,4경기는 준플레이오프 승리팀 홈구장에서 열린다.  \n   \n2024년 프로야구 플레이오프 일정은 다음과 같다.\n\n 10월 13일 (일) 14:00 - 대구\n 10월 14일 (월) 18:30 - 대구\n 10월 16일 (수) 18:30 - 준플레이오프 승리 팀 홈구장\n 10월 17일 (목) 18:30 - 준플레이오프 승리 팀 홈구장\n 10월 19일 (토) 14:00 - 대구\n\n2024년 9월 17일 기준으로 플레이오프 경기는 두산, KT, LG 중 한 팀과 삼성이 하게 될 것이 확실시된다.\n\n### 프로야구 2024 포스트시즌 진

In [22]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 SNS 게시글 형태로 100자 내외로 작성하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
2024년 프로야구 플레이오프 진출 팀은 KIA, 삼성, LG, 두산, KT입니다! KT는 와일드카드 결정전에서 두산을 이기고 준플레이오프에 진출했어요! ⚾️🔥 #프로야구 #플레이오프


In [23]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변에 한국 시리즈 일정을 추가하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024 한국 시리즈 일정
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024 한국 시리즈 일정'}`



[관찰 내용]
Observation: [{'title': '2024 KBO 한국시리즈 일정 및 예매 일정 정보 - 나아가다 - 티스토리', 'url': 'https://can-better.tistory.com/89', 'content': "본문 바로가기\n\n# 나아가다\n\n관심 주제\n\n# 2024 KBO 한국시리즈 일정 및 예매 일정 정보\n\n by Forward-Man 2024. 10. 19.\n\n포스트시즌 플레이오프 경기가\n\n삼성의 승리로 마무리 되었습니다.\n\n그로인해,\n\n먼저 한국시리즈 직행을 확정지은 KIA의 상대가 정해지면서\n\n한국시리즈 일정이 확정되었습니다.\n\n포스팅 목차\n\n### 2024 한국시리즈 일정\n\n1차전 10/21 (월)\n\n 장소 : 광주\n\n 시간 : 18시 30분\n\n2차전 10/22 (화)\n\n 장소 : 광주\n\n 시간 : 18시 30분\n\n3차전 10/24 (목)\n\n 장소 : 대구\n\n 시간 : 18시 30분\n\n4차전 10/25 (금)\n\n 장소 : 대구\n\n 시간 : 18시 30분\n\n5차전 10/27 (일)\n\n 장소 : 광주  \n 시간 : 14시 00분\n\n6차전 10/28 (월)\n\n 장소 : 광주\n\n 시간 : 18시 30분\n\n7차전 10/29 (화)\n\n 장소 : 광주\n\n 시간 : 18시 30분 [...] ### 2024 한국시리즈 예매 정보\n\n \n\n \n\nKBO는 10월 21일(월)부터 개최되는 2024 신한 SOL뱅크 KBO 한국시리즈의 입장권 예매를 20일(일), 21일(월) 이틀에 걸쳐 진행한다.\n\n한국시리즈 입장권은 포스트시즌 입장권 단독 판매사인 인터파크의 검색창